# Grafové databáze
Grafové databáze pro reprezentaci dat nepoužívají tabulky, nýbrž nody (vrcholy) a vztahy mezi nimi (relationship, de facto hrany/edge). Jsou užitečné pro práci s komplikovanými vztahy mezi entitami, které by se u klasických relačních databází musely řešit x-násobnými joiny. Jednou z nejznámějších databází tohoto typu je Neo4j. Toto povídání je zaměřeno na zacházení s její desktopovou verzí, která se dá stáhnout [zde](https://neo4j.com/download/).  
## Obsah
- [Spuštění](#Spuštění)
- [Základní příkazy](#Základní-příkazy)
- [Načítání dat z csv souboru](#Načítání-dat-z-csv-souboru)
- [Dotazy nad složitějšími daty](#Dotazy-nad-složitějšími-daty)
- [Data science a grafy](#Data-science-a-grafy)
  - [Pluginy](#Pluginy)
  - [In-memory grafy](#In-memory-grafy)
  - [Hledání nejkratší cesty](#Hledání-nejkratší-cesty)
  - [Degree centrality](#Degree-centrality)
  - [Closeness centrality](#Closeness-centrality)
  - [Betweenness centrality](#Betweenness-centrality)
  - [Page rank](#Page-rank)
  - [Triangle count](#Triangle-count)
  - [Clustering koeficient](#Clustering-koeficient)
  - [Strongly connected components](#Strongly-connected-components)
  - [Weakly connected components](#Weakly-connected-components)
  - [Label propagation](#Label-propagation)
  - [Louvain modularity](#Louvain-modularity)

## Spuštění
Po naběhnutí Neo4j vlezme nejprve do nastavení (ozubené kolo v levém panelu) do sekce "Data path". Zde se v textovém poli nalézá "root" projektů v Neo4j. To jo pro nás bude důležité zejména v okamžiku, kdy do Neo4j začneme nalévat data z externích souborů. Ty totiž budou muset být někde v tomto adresáři (resp. jeho poadresářích). Důvodem je securita (aby Neo4j nevidělo někam, kam nemá). Pozor - pokud tento adresář změníme, některé projekty nebudou fungovat (asi mají v nastavení starou cestu, která se musí ručně přepsat).  
![settings_data_path](neo4j_figures/settings_data_path.png)
Vyrobme nyní nový projekt (ikona adresáře nalevo nahoře) a pro přehlednost ho rovnou přejmenujme z defaultního "Project" na něco rozumnějšího (přejmenování realizujeme najetím na název a kliknutím na objevivší se ikonu). Přidejme do projektu databázi (tlačítko "Add" -> "Local DBMS"). Zde zdůrazněme, že se opravdu jedná o databázi a ne databázový kontejner. Tj. nejde přistupovat v rámci jedné databáze do databáze druhé - vlastně ani není možné, aby najednou běžely dvě databáze současně. Když totiž spustíme druhou, první se automaticky zavře.
![new_project](neo4j_figures/new_project.png)
Dovnitř databáze se dostaneme kliknutím na ní a poté na modré tlačítko "Open".
![info_database](neo4j_figures/into_database.png)
Pozn: pokud vám spuštění databáze zfailovalo, zkontrolujte, zda něco (např Jupyter notebook) náhodou neblokuje porty.

## Základní příkazy
Modus operandi práce v právě otevřeném Neo4j browseru spočívá v tom, že člověk napíše kód do určité buňky a obsah buňky potom spustí pomocí modrého trojúhelníku. Princip buněk je podobný jako u Jupyter notebooku - lze je spouštět napřeskáčku, přičemž výsledek práce jedné buňky může ovlivnit buňky ostatní. Oproti Jupyteru tu ale můžeme pozorovat jeden rozdíl - buňka se chová jako transakce. Tj. pokud běh buňky spadne na třetím příkazu, výsledky prvních dvou příkazů se revertují. Zdůrazněme nakonec, že použitý jazyk, ve kterém se tyto příkazy píší, už není SQL, nýbrž Cypher.  
Zobrazme si nejprve všechny nody v databázi. To provedeme pomocí
```
match(n) return n
```
Match říká, jaký pattern v databázi hledáme a return pak výsledke matche vrátí do browseru. Písmeno n sice značí node, ale to je jen pomůcka pro čtenáře - klidně by tu místo "n" mohlo být "tralala" a výsledek by byl úplně stejný. Pravda, v současnosti výsledkem jedno velké nic. Zkusme to napravit - vytvořme node pomocí
```
create (n)
```
Když nyní znovu spustíme "matchovací" příkaz, už nějaký výstup uvidíme. Panelem v levé části buňky lze překlikávat mezi jednotlivými reprezentacemi výstupu, vizuálně nejzajímavější bude ale asi ta první - graf.
![match_anonymous_node](neo4j_figures/match_anonymous_node.png)
Když nyní znovu spustíme "create (n)", neobjeví se žádný error ani warning - zkrátka se vytvoří nový node identický s tím předchozím. Když se obou nodů budeme chtít zbavit, použijeme
```
match(n) delete (n)
```
Vytváření anonymních nodů by moc užitečné nebylo. Když chceme použít node určitého typu, realizujeme to pomocí dvojtečky a názvu typu nodu (tzv. labelu), tj. takto:
```
create (n:Žirafa)
```
Z toho mimo jiné vidíme, že Neo4j úspěšně popasuje s českými znaky. Samozřejmě je otázkou, zda je i tak dobrý nápad české znaky používat - spíš ne.  
Výše ukázaným postupem jsme vytvořili node určitého typu. Počítá se s tím, že typ bude sdílet hromada nodů. Když potřebujeme do nodu přidat informaci, která bude vlastní jen nodu jednomu, použijeme tzv. property (vlastnosti). Ty se vytvoří pomocí složených závorek:
```
create (n:Žirafa{name:'Bětka'})
```
Properties je možné mít v jednom nodu více - při jejich zápisu je oddělíme čárkami. Co se týče uvozovek označující začátek a konec textových řetězců, ty mohou být jak jednoduché, tak dvojité:
```
create (n:Žirafa{name:"Dlouhokrčka", height_in_m:5.2})
```
Vytvořme pár nodů jiného typu. Z následujícího je vidět, že v jedné buňce browseru může být naráz více příkazů. jen musí mít odlišnou proměnnou označující node.
```
create (n1:Jídlo{name:"Jablko"})
create (n2:Jídlo{name:"Listy akácií"})
create (n3:Jídlo{name:"Hamburger"})
```
Na nody se můžeme podívat pomocí již zmíněného "match (n) return n".
![giraffe_1](neo4j_figures/giraffe_1.png)
Pokud bychom měli v databázi přiliš mnoho nodů a chtěli zobrazit jen pár z nich, abychom vlastně pochopili, jak vypadají, můžeme použít klíčové slovo limit:
```
match(n) return n limit 3
```
Vytvořme nyní mezi některými nody vztahy. To se udělá následujícím způsobem (pokud příkaz nekopírujete, ale ručně píšete, tak odřádkování dosáhnete pomocí SHIFT+ENTER):
```
match(z:Žirafa), (j:Jídlo)
where z.name = 'Bětka' and j.name = 'Jablko'
create (z)-[rel:má_ráda]->(j)
``` 
Co se tady vlastně děje? Na prvním řádku říkáme, že chceme nody typu Žirafa a Jídlo. Na řádku druhém klademe na nody určité podmínky - využíváme vlastností nodů, ke kterým přistupujeme prostřednictvím tečkové notace. Na řádku třetím vytváříme samotný vztah mezi nody. Ten je definovám v hranatých závorkách, přičemž "rel" je zastupující proměnná a "má_ráda" jméno vztahu. Pro označení nodů, mezi kterými vztah platí, použijeme kulaté závorky a jejich zastupující proměnnou. Pomlčka a šipka pak určují, kterým směrem vztah jde.  
Výstupem podmínky může být pochopitelně i více než pouze jeden node od každého typu:
```
match(z:Žirafa), (j:Jídlo)
where j.name = 'Listy akácií'
create (z)-[rel:má_ráda]->(j)
```
Vztahy mohou platit i mezi nody stejného typu:
```
match(z1:Žirafa), (z2:Žirafa)
where z1.name = 'Bětka' and z2.name = 'Dlouhokrčka'
create (z1)-[rel:je_matkou]->(z2)
```
![giraffe_rel](neo4j_figures/giraffe_rel.png)
Položme nyní databázi pár dotazů. Nejprve bychom chtěli znat všechny žirafy, které mají rády jablka.
```
match (z:Žirafa)-[:má_ráda]->(:Jídlo {name:"Jablko"}) return z
```
Všimněte si, že pro nody, které dále nebudeme potřebovat, nemusíme ani psát zastupující proměnnou.
![giraffe_likes_apple](neo4j_figures/giraffe_likes_apple.png)
Naopak když bychom chtěli znát všechna jídla, která má žirafa Bětka ráda, napíšeme
```
match (:Žirafa {name:"Bětka"})-[:má_ráda]->(j:Jídlo) return j
```
![giraffe_likes_apple_2](neo4j_figures/giraffe_likes_apple_2.png)
Dotaz může vracet i více typů nodů naráz. Například když bychom u výše uvedeného dotazu chtěli mít v grafu i žirafu, vyrobili bychom pro ni zástupnou proměnnou, která by se poté objevila v return sekci:
```
match (z:Žirafa {name:"Bětka"})-[:má_ráda]->(j:Jídlo) return z,j
```
![giraffe_likes_apple_3](neo4j_figures/giraffe_likes_apple_3.png)

Poznamenejme, že node může mít na sebe nalepeno více labelů už od svého vzniku - stačí při jeho vytváření přidat další dvojtečky a typy:
```
create (n:Žirafa:Zvíře)
``` 
Co musíme udělat, když chceme data v grafové databázi upravit? Použijeme klíčové slovo SET. Například pokud bychom chtěli Bětku přejmenovat na Hortenzii, napíšeme následující:
```
match (z:Žirafa {name:"Bětka"})
set z.name = "Hortenzie"
return z
```
Všimněte si, že při úpravě property jsme použili v setu rovnítko. Když budeme chtít přidat k nodu nový label, použijeme namísto toho dvojtečku:
```
match (z:Žirafa {name:"Dlouhokrčka"})
set z:Savec
return z
```
Zdůraňuji, že výše uvedený příkaz label k nodu přidal, tj. nový label nenahradil label stávající. Pro odstranění labelu použijeme klíčové slovo remove:
```
match (z:Žirafa {name:"Dlouhokrčka"})
remove z:Savec
return z
```
Co ale udělat, když bychom chtěli přejmenovat "anonymní" žirafu? Té jsme žádný parametr, na kterých bychom se mohli v matchování chytit, nedali. I přesto ale naštěstí jeden parametr má - id. Bohužel je ale práce s id složitější - příkaz typu "match(n {id:1165}) return n" nic nevrací. Zde musíme použít where konstrukci a id(node) = id_číslo:
```
match(n) 
where id(n) = 1165
set n.name = "Silvestr"
return n
```
Nakonec ještě zmiňme, že u víceřádkových dotazů/příkazů může být užitečné jejich část zakomentovat - to se provádí pomocí dvojitého dopředného lomítka, tj. //.  
Na závěr kapitoly databázi promažeme. Když vyzkoušíme výše uvedené "match(n) delete (n)", vrátí se nám chyba "Neo.ClientError.Schema.ConstraintValidationFailed" s popiskem "Cannot delete node<0>, because it still has relationships. To delete this node, you must first delete its relationships.". Jelikož jsme ale líní a nechceme nejprve smazat vztahy a až potom nody, použijeme
```
match(n) detach delete (n)
``` 

## Načítání dat z csv souboru
Zkusme do Neo4j nalít data ze souboru animals.csv, který najdete v tomto repozitíři ve složce "tutorial_data". Naivním přístupem by bylo vložení nášeho souboru do "Data path" adresáře uvedeného v nastavení a posléze spuštění buňky obsahující
```
load csv with headers from
'file:///animals.csv' as csv
FIELDTERMINATOR ';'
create (anim:Animal {animal:csv.animal, eats:csv.food})
```
Pak bychom dostali chybovou hlášku typu "Neo.ClientError.Statement.ExternalResourceFailed" obsahujíc zprávu "Couldn't load the external resource at: file:/C:/vs/programy/poznamky_neo4j/relate-data/dbmss/dbms-bfd104f5-3c9c-474e-bc61-e99686446da7/import/animals.csv". Zde "C:/vs/programy/poznamky_neo4j" je můj "Data path" adresář. Nezbývá než soubor vložit do adresáře, kde ho Neo4j chce mít.  
Co se ale v příkazu vlastně děje? Na prvním řádku říkáme, že chceme načíst csvčko s hlavičkou. Druhý řádek prozrazuje, odkud a do jaké proměnné se má obsah souboru uložit. Třetí řádek stanovuje separátor polí - pokud je jím v souboru čárka, tak se fieldterminator udávat nemusí. Nakonec čtvrtý řádek říká, že se pro každý řádek z csv souboru má vytvořit node Animal, který má mít jednak parametr animal načítaný ze sloupce animal, jednak parametr eats načítaný ze sloupce food.  
Když to uděláme, dost možná se nám zobrazí pět nodů nadepsaných jídlem. Jenže my bychom chtěli mít nody nadepsané zvířetem. Pro dosažení tohoto cíle musíme kliknout na zobáček (<) v pravého horním rohu buňky, následně kliknout na chtěný typ nodu (zde Animal) a poté vybrat na řádku "Caption" chtěný parametr.  
![bad_top_property](neo4j_figures/bad_top_property.png)
Jenže je zde další problém. Ve vstupních datech byl jeden řádek (cow) zdvojen. Naštěstí zde pro opravu nemusíme dělat žádný preprocessing. Stačí když spustíme (po promazání databáze) načítací příkaz, kde "create" zaměníme za "merge".
```
load csv with headers from
'file:///animals.csv' as csv
FIELDTERMINATOR ';'
merge (anim:Animal {animal:csv.animal, eats:csv.food})
```
Všimněme si, že odstraněn byl jen zcela duplicitní řádek. Žirafy máme pořád dvě, protože každá jí něco jiného. Situace je ale komplikovanější. Merge se totiž fakticky pokusí o match a až když s tím neuspěje, tak vytvoří nový node. Co to fakticky znamená? Kdybychom měli 
```
create (pac1:Balíček {name:"Koala", version:1})
create (pac2:Balíček {name:"Koala"})
```
vytvořily by se nám dva nody, z nichž jeden by měl a jeden by neměl parametr version. Když ale použijeme kód
```
create (pac1:Balíček {name:"Koala", version:1})
create (pac2:Balíček {name:"Koala"})
```
vznikne jenom jeden node, přičemž parametr version bude mít. Na prvním řádku se totiž začne hledat balíček o daném jménu a verzi, nebude nalezen a tak se založí. Na řádku druhém se bude hledat balíček o daném jméně (verze je nyní irrelevantní), nalezen bude a tak se druhý node už nevytvoří.  
K čemu je to dobré? Za chvíli načteme pomocí příkazu 
```
load csv with headers from
"file:///pandas_required_packages.csv" as csv
FIELDTERMINATOR ';'
merge (pac1:Package {name:csv.name, version:csv.version, license:csv.license, summary:csv.summary})
merge (pac2:Package {name:csv.required_packages})
create (pac1)-[spoj:REQUIRES {extra_flag:csv.extra_flag}]->(pac2)
```
data ze souboru pandas_required_packages.csv (k nalezení stejně jako animals.csv ve složce tutorial_data). Tento soubor má strukturu typu
```
name;version;license;summary;required_packages
pandas;1.3.5;BSD-3-Clause;Powerful data structures for data analysis, time series, and statistics;python-dateutil
pandas;1.3.5;BSD-3-Clause;Powerful data structures for data analysis, time series, and statistics;hypothesis
pandas;1.3.5;BSD-3-Clause;Powerful data structures for data analysis, time series, and statistics;pytz
hypothesis;6.31.4;MPL v2;A library for property-based testing;tzdata
hypothesis;6.31.4;MPL v2;A library for property-based testing;pytz
``` 
Pokud bychom měli namísto merge create, vedl by řádek s "(pac2:Package {name:csv.required_packages})" u příkladu dat k vytvoření dodatečného nodu, který bychom nechtěli.  
Potížím ale ještě není konec. Některé balíčky totiž žádné prerekvizitní balíčky nepožadují, tj. v poli required_packages nemají nic. Toto "nic" neo4j interpretuje jako null hodotu. Jenomže merge s null hodnotami nedokáže pracovat. Proto jsem musel na takováto místa vložit neexistující balíček "fake_package". Marge tak úspěšně doběhne, ale v databází nyní máme nesmyslný node. Ten vymažeme pomocí
```
match (pac3:Package {name:"fake_package"}) detach delete pac3
```
Pokud bychom chtěli spustit oba dva příkazy najednou v jedné buňce, musíme je oddělit středníkem.

## Dotazy nad složitějšími daty
Nad z csv souboru načtenými daty si vyzkoušíme některé lehce složitější dotazy. Napřed bychom si ale měli říct, co v datech vlastně je. Jedná se o rekurzní závislosti pythoního balíčku Pandas. Pokud jste někdy tento balíček instalovali ručně, určitě vás překvapil podezřelý počet řádků v csv souboru. To je dané zdrojem dat - API pypi serveru alias stránkami typu https://pypi.org/pypi/pandas/json. Když kliknete na odkaz a podíváte se do sekce info -> requires_dist, uvidíte tam krom nezbytných pytz, python-dateutil a numpy i hypothesis, pytest a pytest-xdist, tj. balíčky, které člověk potřebuje jen při vývoji. Ty jsem neodfiltroval ani u pand, ani u žádného dalšího rekurzivně zkloumaného balíčku. Tím pádem soubor nakynul, ale to pro naše "zkušební" potřeby to je asi jenom dobře. Až budeme chtít "nedefaultní" balíčky odstranit, použijeme sloupec, resp. property vztahů extra_flag - právě parametr extra na výše uvedené stránce určuje prerekvizitní balíčky, které se samy defaultně nenainstalují.  

Chtějme nejprve ukázat všechny balíčky, které Pandas bezprostředně vyžaduje. Navíc si zobrazme i samotný balíček Pandas:
```
match (pac0:Package {name:"pandas"})-[:REQUIRES]->(pac:Package ) return pac0, pac
```
Nyní se zeptejme na všechny balíčky, které jsou ob jeden balíček pandami vyžadovány. Zde využijeme hvězdičky umístěné do vztahu, přičemž vedle ní napíšeme požadovaný počet skoků od pand.
```
match (:Package {name:"pandas"})-[:REQUIRES*2]->(pac:Package ) return pac
```
A co kdybychom chtěli jak bezprostředně vyžadované balíčky, tak balíčky o krok dál? V takovém případě za hvězdičku napíšeme spodní okraj uzavřeného intervalu s vyžadovaným počtem skoků, dvě tečky a horní okraj uzavřeného intervalu s vyžadovaným počtem skoků:
```
match (:Package {name:"pandas"})-[:REQUIRES*1..2]->(pac:Package ) return pac
```
Explicitně jsme zda zobrazení balíčku pandas nevyžadovali, ale přece se nám objevil - závisí na něm nějaký jiný balíček. Když takovéto chování nechceme, zakážeme ho pomocí where podmínky. Zde si všimněme, že nerovnost se zapisuje jako <>.
```
match (pan:Package {name:"pandas"})-[:REQUIRES*1..2]->(pac:Package ) 
where pan <> pac
return pac
```
Výstup pravda moc přehledný není. Řešením je vracet nikoli celý node, ale pouze jednu jeho property - jméno:
```
match (pan:Package {name:"pandas"})-[:REQUIRES*1..2]->(pac:Package ) 
where pan <> pac
return pac.name
```
Zmiňme nakonec, že pokud bychom chtěli závislosti do libovolné hloubky, nepsali bychom za hvězdičku žádné číslo. Když jsem ale u sebe takovýto dotaz odpálil, nezdálo se, že by query chtěla doběhnout v rozumném čase. Nicméně když provedeme defaultně nestahované balíčky odfiltrujeme, dotaz doběhne hned:
```
match (pan:Package {name:"pandas"})-[:REQUIRES* {extra_flag:"False"}]->(pac:Package ) 
return pan, pac
```
Co kdybychom chtěli znát všechny balíčky, na kterých přímo závisí pandas, ale které na žádném dalším balíčku nezávisí? Tehdy opět použijeme where podmínku. V ní ale uplatníme exists (čí přesněji not exists) a kontrolu existence vztahu typu REQUIRES:
```
match (:Package {name:"pandas"})-[:REQUIRES]->(pac:Package ) 
where not exists ( (pac)-[:REQUIRES]->(:Package))
return pac
```
Podle očekávání se nám vrátí jen pytz a numpy.  

Co když nechceme znát nody samotné, ale zajímá nás jen jejich počet? Tehdy nevrátíme jejich zastupující proměnou, ale count(zastupující proměnná):
```
match (:Package {name:"pandas"})-[:REQUIRES]->(pac:Package ) return count(pac)
```
Vidíme, že výsledek se nyní neukáže v grafové záložce, ale v tabulkové záložce. Nemuselo by se nám líbit, že se sloupeček s počtem nodů jmenuje "count(pac)". To lze vyřešit klíčovým slovem "as" a přejmenováním sloupce:
```
match (:Package {name:"pandas"})-[:REQUIRES]->(pac:Package ) return count(pac) as pocet_potrebnych
```
Podobně jako count fungují i jiné agregační funkce, např. avg, min či max. Odlišný je fakt, že tyto funkce budou mít v argumentu nikoli node, ale property nodu. Zde si neukážeme příklad na datech, neboť u grafu balíčků to nedává smysl. Nicméně mohli bychom si to představit jako avg(cosi.height).
Neo4j můžeme říct, aby vrátilo nody seřazené podle nějaké property - to zrealizujeme pomocí klauzule "order by" umístěného za returnem. Samozřejmě v grafové sekci buňky neuvidíme žádnou změnu - pro to se musíme překliknout do sekce "Table". Defaultní řazení je vzestupné; pokud chceme opak, musíme to explicitně zmínit pomocí klíčového slova desc. 
```
match (:Package {name:"pandas"})-[:REQUIRES]->(pac:Package) return pac.name as package_name order by pac.name desc
```

## Data science a grafy
#### Pluginy
Doposud jsme nad daty v databázi spouštěli jen relativně jednoduché příkazy. Existují ale i komplikovanější konstrukce, často spojené s data science. Psát je ručně od podlahy by vedlo k hromadě překlepů a netriviálních chybových hlášek, naštěstí je ale za nás už někdo dřív dostal do procedur. Tyto procedury nicméně nejsou součástí základního Neo4j - musí se do něj dostat v pluginech.  
Pro instalaci pluginů klepneme v základním Neo4j rozhraní na náš projekt a následně na databázi. Objeví se nový panel, ve kterém vstoupíme do záložky plugin. Zde rozklikneme "APOC" a "Graph Data Science Library" a u obou zmáčkneme tlačítko "Install". A co jsme to vlastně nainstalovali? APOC (aka Awesome Procedures on Cypher) je sbírka nejrůznějších utilit. Graph Data Science Library (aka GDS) zase obsahuje algoritmy spojené s data science.  
![plugins](neo4j_figures/plugins.png)  
Jako příklad pěkné APOCí utility zmiňme zobrazení schématu. Procedura se spustí pomocí
```
call db.schema.visualization
```
![schema](neo4j_figures/schema.png)  
#### In-memory grafy
Pro práci s data science procedurami bude užitečné si vytvořit in-memory graf. De facto se jedná o pohled nad v databázi uloženými daty, který obsahuje jen pro naše potřeby relevatní nody a vztahy. Jelikož je in-memory, tak se při vypnutí Neo4j ztratí. Vytvoříme ho následujícím předpisem:
```
call gds.graph.create.cypher(
    "package_graph",
    "match (pac1:Package) return id(pac1) as id",
    "match (pac1:Package) - [:REQUIRES] -> (pac2:Package) return id(pac1) as source, id(pac2) as target"
)
yield graphName as graph, nodeQuery, nodeCount as nodes, relationshipQuery, relationshipCount as rels
```
První parametr procedury je jméno in-memory grafu. Druhý parametr obsahuje query na nody(tj. musí tam být match ... return ...), která musí vracet id nodů. Přesněji řečeno - výstupem musí být sloupec pojmenovaný id. Ve třetím parametru je query na vztahy, která také musí vracet dvoje idčka nodů pojmenované jako source a target. Zdůrazněme, že tyto parametry nejsou volitelné. Všimněme si, že výstup procedury je uvozen klíčovým slovem yield a nikoli return. Jenže bacha - pokud bychom chtěli například výstup seřadit pomocí order by, dostali bychom chybovou hlášku. Tudíž bývá obvykle lepší za yield dát return řádek s dodatečnými operacemi.    
Chceme-li se podívat na už zadefinované in-memory grafy, použijeme 
```
call gds.graph.list()
```
Snaha znova vytvořit graf o stejném jméně skončí errorem "A graph with name 'package_graph' already exists.". Tj. když zjistíme, že jsme při vytváření grafu udělali chybu, musíme původní graf napřed vymazat. To provedeme pomocí příkazu gds.graph.drop, který jako parametr přebírá právě jmnéno odstraňovaného balíčku:
```
call gds.graph.drop("package_graph")
```  

Než se dostaneme k samotným DS procedurám, musím zmínit, že jsem zde prezentované informace čerpal z publikace "Graph Algorithms: Practical Examples on Apache Spark and Neo4j" (autoři M. Needham a A. E. Hodler, nakl. O'Reilly, 2019).  
#### Hledání nejkratší cesty
Podívejme se nejprve na to, jak najít nejkratší cestu mezi dvěma nody. To realizujeme následující konstrukcí:
```
match (source:Package {name: "pandas"}), (target:Package {name: "six"})
call gds.shortestPath.dijkstra.stream(
  "package_graph",
  {
    sourceNode: source,
    targetNode: target
  }
)
yield index, sourceNode, targetNode, totalCost, nodeIds, costs, path
return index,  sourceNode,  targetNode,  totalCost,  nodeIds,  gds.util.asNodes(nodeIds), costs, path
```
Vidíme, že se zde nejprve pomocí matche vytvoří proměnné source a target obsahující počáteční a koncový node. Následně se zavolá procedura gds.shortestPath.dijkstra.stream. Ta jako parametry přebírá jednak jméno in-memory grafu, jednak konfigurační mapu (tj. to ve složených závorkách), ve které proceduře počáteční a koncový node předáváme. Co nám procedura vrací? V případě, že cesta neexistuje (například mezi nody není žádnéé spojení), tak se nevrátí nic. Co ale když spojení mezi nody existuje? Tehdy dostaneme jednořádkovou tabulku se sloupci definovanými v returnu. Index je indexem nalezené cesty - my tu máme jednu cestu, takže to bude nula. SourceNode a targetNode jsou idčka počátečního a koncového nodu. TotalCost udává počet nodů v cestě. Jelikož má cesta minimálně počáteční a koncový node, je totalCost alespoň dva. NodeIds obsahuje list idček nodů v cestě. Jelikož ty samy o sobě příliš neřeknou, přidáme do return sekce asNodes funkci, která list idček transformuje na list nodů. Costs reprezentuje kumulativní cenu cesty. V našem případě jsme vztahům (spojnicím nodů) cenu nedefinovali. Tudíž každý skok mezi nody stojí jednu jednotku, což vzhledem k existenci totalCostu moc užitečné není. Nicméně pokud bychom hledali cestu například mezi dvěma městy na mapě, už by to užitečné bylo - tehdy by totiž spojení měly každé jinou cenu. Nakonec path obsahuje cypherovský objekt cestu reprezentující.  
Obsah sloupce nodeIds je poněkud nepřehledný - lepší by bylo, kdybychom viděli pouze seznam jmen balíčků a ne seznam nodů s veškerým příslušenstvím. To se dá zařídit zápisem trochu připomínající pythonovské list comprehension:
```
match (source:Package {name: "pandas"}), (target:Package {name: "six"})
call gds.shortestPath.dijkstra.stream(
  "package_graph",
  {
    sourceNode: source,
    targetNode: target
  }
)
yield index, sourceNode, targetNode, totalCost, nodeIds, costs, path
return index, sourceNode, targetNode, totalCost, nodeIds, [one_node in gds.util.asNodes(nodeIds)| one_node.name] as node_names, costs, path
```  
#### Degree centrality
Dále se podívejme na tzv. centrality algoritmy. Jendá se o algoritmy, které se používají na pochopení role určitých nodů v grafu. Nejjednodušší z nich je degree centrality. Jedná se o počet vztahů (vstupních i výstupních), které jsou na node napojeny.
```
call gds.degree.stream("package_graph") yield  nodeId, score
return gds.util.asNode(nodeId).name as name, score
order by score desc limit 5
```
Příslušná procedura má jen jeden povinný parametr - jméno in-memory grafu. Vrací ID nodů s odpovídajícím score (počtem vztahů). Jelikož ID samotné by nám nic moc neřeklo, převádíme ho v rámci return sekce na odpovídající node, resp. jednu jeho property - jméno. Dál pak nody řadíme podle score a necháváme si vypsat jen prvních 5.  

| name | score |
| - | - |
|"pyobjc"|278.0|
|"panel"|51.0|
|"holoviews"|49.0|
|"hvplot"|40.0|
|"celery"|40.0|

#### Closeness centrality
Degree centrality je nejzákladnější centrality algoritmus. Mezi trochu složitější postupy patří closeness centrality. Ta je pro konkrétní node definovaná jako invertovaná průměrná vzdálenost od všech ostatních nodů. Tj. platí pro ni vzorec  
$C(u) = \frac{1}{\sum_{v=1}^{n-1}{d(u,v)}}$  
kde **C(u)** znaší closeness centralitu pro node **u**, **n** je počet nodů v grafu a **d(u,v)** nejkratší cesta mezi nodem **v** (odlišným od nodu **u**) a nodem **u**.  
Obvykle se centralita normalizuje, aby neukazovala sumu nekratších cest, ale jejich průměrnou délku:  
$C(u) = \frac{n-1}{\sum_{v=1}^{n-1}{d(u,v)}}$  
Takto je to naimplementováno i v Neo4j.  
Použití procedury vypadá takto:
```
call gds.alpha.closeness.stream("package_graph")
yield nodeId, centrality
return gds.util.asNode(nodeId).name as name, centrality
order by centrality desc limit 5
```
Výstupem je  

|name|centrality|
|-|-|
|"pytest"|1.3060109289617485|
|"sphinx"|1.07078853046595|
|"pytest-cov"|1.0491659350307287|
|"requests"|1.0328435609334485|
|"importlib-metadata"|0.9851607584501236|

Vidíme, ža narozdíl od degree centrality tu už vidíme známé balíčky. Nejlepší score zde má balíček na psaní testů pytest následovaný balíčkem na generování dokumentace sphinx. To dává docela smysl - využívá je (napřímo) hromada balíčků a tudíž mají v průěru ve všem ostatním balíčkům blízko. Naproti tomu balíček pyobjc, který měl největší degree centrality, je sice napojen na hromadu určitých balíčků, ale k balíčkům ostatním to už má daleko.  

#### Betweenness centrality
Jinou metrikou je betweenness centrality. Pro její výpočet se nejprve najde nejkratší cesta mezi každou dvojicí nodů v grafu. Následně každý node získá score odpovídající tomu, v kolika nejkratších cestách figuroval. Tj. platí vzorec  
$B(u) = \sum_{s\neq u\neq t}\frac{p(u)}{p}$  
Zde **B(u)** je betweenness centrality pro node **u**, **p** značí počet nejkratších cest mezi nody **s** a **t**, **p(u)** je počet nejkratších cest mezi **s** a **t**, které obsahují **u**.  
Použití v Neo4j:
```
call gds.betweenness.stream("package_graph")
yield nodeId, score
return gds.util.asNode(nodeId).name as name, score
order by score desc limit 5
```
Výsledek:

|name|score|
|-|-|
|"furo"|183887.0973896277|
|"setuptools"|176475.59472524203|
|"pydata-sphinx-theme"|166355.7164286598|
|"xarray"|163441.84110646194|
|"pytest"|153962.78438724807|

Zde v kontextu pythoních balíčků jen stěží hledám vhodnou interpretaci. Kdyby ale nody obsahovaly uživatele Facebooku/Linkedinu, šlo by o osoby propojující různé skupiny lidí.  

#### Page rank
Když už jsme u algoritmů, které se budou vzhledem k našim datům možná obtížně interpretovat, podívejme se na PageRank. Ten byl vyroben v Googlu, aby řadil stránky ve vyhledávači. Princip spočívá v předpokladu, že stránka, na kterou se odkazuje víc relevantních stránek, bude také relevantní. V kontextu Neo4j a našich dat se zde tak neměří přímo vliv jednotlivého nodu, ale vliv sousedů (a sousedů sousedů) onoho nodu.  
Mohli bychom se ptát, jak algoritmus pozná vliv sousedů. Odpovědí je fakt, že k tomu algoritmus dojde iteračně. Na začátku má každý node váhu 1/počet nodů. Pak se spočítá váha outgoing vztahů jako hodnota nodu dělená počtem outgoing vztahů. Nová váha nodů pak bude tvořit součet vah všech incoming vztahů do nodu mířících. Toto se ještě n-krát zopakuje, kde n je počet iterací stanovených uživatelem. Defaultně je tento maximální počet iterací roven 20, ale můž být i menší, pokud jsou změny vah všech nodů mezi jednotlivými iteracemi pod prahovou hodnotou.    
```
call gds.pageRank.stream("package_graph")
yield nodeId, score
return gds.util.asNode(nodeId).name as name, score
order by score desc limit 5
```
|name|score|
|-|-|
|"pyobjc-core"|16.4156002826566|
|"pytest"|10.556888539132432|
|"pyobjc-framework-Cocoa"|8.472847728030414|
|"six"|6.110718437369952|
|"sphinx"|4.654555298166597|

Vidíme, že mezi top 5 balíčky z hlediska pageranu je pytest, sphinx a six. To dává smysl v tom duchu, že pro celý ekosystém nejdůležitější balíčky potřebují být otestovány, musí mít dokumentaci a musí mít zajištěnou kompatibility mezi Pythonem 2 a 3.

#### Triangle count
Podívejme se nyní na community detection algoritmy. S jejích pomocí nalezneme komunity nodů, tedy skupiny nodů, které mají mezi sebou víc vztahů než s nody mimo ony skupiny.  
Nejprve se podívejme na tzv. triangle count. Trojúhelník v tomto kontextu představuje skupinu tří nodů, z nichž každý má vztah se zbylými dvěma. Jedná se tak o komunitu nodů. Triangle count pro určitý node je pak zkrátka počet trojúhelníků, u kterých onen node představuje jeden z jejich bodů. Tuto veličinu získáme následujícím kódem:
```
call gds.triangleCount.stream("package_graph")
yield nodeId, triangleCount
return gds.util.asNode(nodeId).name as name, triangleCount
order by triangleCount desc limit 5
```
Pakliže bychom chtěli toto číslo posčítat přes celý graf, použijeme namísto gds.triangleCount.stream proceduru gds.triangleCount.stats:
```
call gds.triangleCount.stats('package_graph')
yield globalTriangleCount, nodeCount
```
Pokud bychom potřebovali vědět, které nody ony trojúhleníky vlastně tvoří, sáhneme po proceduře gds.alpha.triangles:
```
call gds.alpha.triangles("package_graph")
yield nodeA, nodeB, nodeC
return gds.util.asNode(nodeA).name as first_node_name, gds.util.asNode(nodeB).name as second_node_name, gds.util.asNode(nodeC).name as third_node_name
limit 5
```

#### Clustering koeficient
To je všechno pěkné, ale má výše uvedený algoritmus nějaký jiný účel než odhalení nodů, které nejsou v žádné skupině, i když z nich vychází (resp. do nich vchází) hodně vztahů? Ano, má - triangle count se dá použít při výpočtu clustering koeficientu. Ten nám říká, jaká je pravděpodobnost toho, že sousedi nodu jsou spolu též spojeni. Clustering koeficient se dá spočítat jako poměr mezi faktickým počtem trojúhelníku a maximálním možným počtem trojúhelníků. Tj. platí pro něj vztah  
$CC(u) = \frac{R_u}{\binom{k_u}{2}} = \frac{2R_u}{k_u(k_u - 1)}$   
Zde **CC(u)** je clustering coefficcient pro node **u**, **R_u** značí počet trojúhlníků, ve kterých node **u** figuruje, **k_u** představuje počet nodů, se kterými má node **u** vztah. Kombinační číslo ve vztahu máme kvůli tomu, že node **u** tvoří jeden bod trojúhelníku a nyní chceme vědět, kolika způsoby lze nalézt další dva body trojúhelníku (s tím, že na pořadí nám nezáleží).  
Kód pro nápočet vypadá takto:
```
call gds.localClusteringCoefficient.stream("package_graph")
yield nodeId, localClusteringCoefficient
return gds.util.asNode(nodeId).name AS name, localClusteringCoefficient
order by localClusteringCoefficient desc
```
Vidíme, že výstupem jsou nesmyslná čísla - koeficient by měl mít význam pravděpodobnosti, tj. měl by se nacházet v intervalu ohraničeném nulou a jedničkou. My zde vidíme nekonečna a za nimi číslo 95. Kde se stala chyba? Problém je způsoben tím, že je daný algoritmus napsán pro neorientované grafy. To ale pro náš graf neplatí. Co s tím? Vytvořme si nejprve neorientovaný graf tak, že vezmeme definici orientovaného grafu, změníme jeho jméno a v sekci vztahů vymažeme zobáček:
```
call gds.graph.create.cypher(
    "package_graph_undirected",
    "match (pac1:Package) return id(pac1) as id",
    "match (pac1:Package) - [:REQUIRES] - (pac2:Package) return id(pac1) as source, id(pac2) as target"
)
yield graphName as graph, nodeQuery, nodeCount as nodes, relationshipQuery, relationshipCount as rels
```
Když tento nový graf vložíme do kódu pro výpočet clustering koeficientu, už dostaneme smysluplná čísla.

#### Strongly connected components
Podívejme se nyní na strongly connected components (SCC). Jedná se o množinu nodů v orientovaném grafu, kde je každý node dosažitelný z každého nodu (tj. je to zacyklený graf, nikoli klasický "jednosměrný" strom). SCC (respektive postupy na jejich odhalení) se hodí pro nalezení skupin, které by si zasluhovaly podrobnější prozkoumání.  
Nejjednodušší kód na rozřazení nodů vypadá takto:
```
call gds.alpha.scc.stream("package_graph")
yield nodeId, componentId
return gds.util.asNode(nodeId).name as Name, componentId as Component
order by Component desc
```
Výstupem je pak tabulka

|Name|Component|
|-|-|
|"pydocstyle"|1195|
|"snowballstemmer"|1194|
|"toml"|1193|
|"enum34"|1191|
|"pycodestyle"|1190|

Zde sloupec "Component" obsahuje idčko množiny nodů, které splňují podmínku SCC. Z pohledu na tabulku se zdá, že většinově mají skupiny po jednom prvku, tj. SCC téměř neexistují. Nicméně je tomu opravdu tak? Abychom na tuto otázku mohli odpovědět, musíme spočítat, u kolika řádků se to které component id opakuje. Jenže jak to realizovat, když se výsledek stream příkazu nikam nezapisuje? Řešením je použití write příkazu, který přidá do nodů novou property - číslo komponenty:
```
call gds.alpha.scc.write("package_graph", {writeProperty: "componentId"})
yield setCount, maxSetSize, minSetSize
```
Už z výstupu tohoto příkazu vidíme, že tu máme množinu o velikosti 503 nodů. Výskyt jednotlivých komponent spočítáme pomocí
```
match (pac:Package)
return pac.componentId as component, count(*) as component_size
order by component_size desc limit 5
```
Nyní vidíme následující:

|component|component_size|
|-|-|
|0|503|
|3|1|
|4|1|
|5|1|
|2|1|

V datech máme jen jednu smyčku, která je ale o to větší.  

#### Weakly connected components
Existuje i termín Weakly Connected Components (WCC). Jedná se o SCC, jen není vyžadována "obousměrnost", tj. stačí, když ze z nodu A dostaneme do nodu B, ale už se z nodu B nemusíme dostat do nodu A. Vzhledem k tomu, jak naše data vznikala (řetěz balíčků vyžadovaných balíčkem Pandas) bychom zde očekávali jednu jedinou WCC množinu.  
Napíšeme tedy 
```
call gds.wcc.write("package_graph", {writeProperty: "componentId_weak"})
yield nodePropertiesWritten, componentCount
```
A ejhle, máme množiny dvě. Podívejme se, kolik nodů se ve které WCC množině nachází, příkazem
```
match (pac:Package)
return pac.componentId_weak as component, count(*) as component_size
order by component_size desc limit 5
```
Výsledek:

|component|component_size|
|-|-|
|0|1196|
|1158|1|

Když se podíváme, co za node to v komponentě 1158 vlastně je, uvidíme, že se jedná o balíček pojmenovaný jako "package_without_link", který jsme vytvořili během předchozích pokusů. Tj. vše funguje tak, jak má.  

#### Label propagation
Jiným způsobem, jak realizovat clustering, je tzv. label propagation (LPA). Principielně jde o to, že se do náhodných nodů grafu umístí seedovací hodnota nové property a ta se šíří dál. V chumlech nodů se bude šířit rychle, na spojnicích chumly spojujících už pomaleji. Počet clusterů si Neo4j určuje samo.
Příklad provedení:
```
call gds.labelPropagation.write("package_graph", { writeProperty: "label_propag_community" })
yield communityCount, ranIterations, didConverge
``` 

#### Louvain modularity
Dalším clusteringovým algoritmem je tzv. Louvain modularity:
```
call gds.louvain.write("package_graph", { writeProperty: "louvain_community" })
yield communityCount, modularity, modularities
```